Vamos a comenzar estableciendo una puntuación en función de las derrotas, victorias y empates teniendo en cuenta la ronda en la que estamos, siendo la fase de grupos la menor ponerada:

Fase de Grupos = Puntos * 0.5
RO16 = Puntos * 0.6
RO8 = Puntos * 0.7
RO4 = Puntos * 0.8
SF = Puntos * 0.9
F = Puntos * 1

Dando 2 puntos por victoria, 1 por empate y -1 por derrota